### Importing libraries

In [94]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [95]:
# URL of the Wikipedia page
# url = "https://en.wikipedia.org/wiki/List_of_career_achievements_by_Cristiano_Ronaldo"
url = "https://en.wikipedia.org/wiki/List_of_career_achievements_by_Lionel_Messi"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all "wikitable" tables on the page
    tables = soup.find_all("table", {"class": "wikitable"})

    # Initialize an empty list to store DataFrames
    dfs = []

    # Process each table and create a DataFrame
    for table in tables:
        # Initialize an empty list to store data
        data = []

        # Extract table data
        for row in table.find_all("tr"):
            row_data = [cell.text.strip() for cell in row.find_all(["td", "th"])]
            data.append(row_data)

        # Create a DataFrame from the data
        df = pd.DataFrame(data)

        # If the DataFrame has more than one row, set the first row as headers
        if len(df) > 1:
            headers = df.iloc[0]
            df = df[1:]
            df.columns = headers

        # Reset the DataFrame index
        df.reset_index(drop=True, inplace=True)

        dfs.append(df)

    # Now, dfs is a list of DataFrames, each containing one "wikitable" section
    # You can access and manipulate these DataFrames as needed.

    # Print the first few rows of each DataFrame
    for i, df in enumerate(dfs):
        print(f"Table {i+1}:")
        print(df.head())
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)


Table 1:
0 Season / year                                   Competition  \
0       2004–05                                       La Liga   
1          2005  South American U-20 Championship Third place   
2          2005                           FIFA U-20 World Cup   
3          2005                           Supercopa de España   
4       2005–06                                       La Liga   

0 Club / national team  Ref.  
0            Barcelona   [4]  
1        Argentina U20   [5]  
2                  [6]  None  
3            Barcelona   [4]  
4                 None  None  
Table 2:
0        Year                       1st                              2nd  \
0        2010  Lionel Messi (Barcelona)       Andrés Iniesta (Barcelona)   
1  Percentage                    22.65%                           17.36%   
2        2011  Lionel Messi (Barcelona)  Cristiano Ronaldo (Real Madrid)   
3  Percentage                    47.88%                           21.60%   
4        2012  Lionel Mes

The Dataframes list is:
- dfs[0]: Collective awards
- dfs[2]: Selections for the best player or forward
- dfs[3]: Top goalscorer
- dfs[4]: Goal of the competition
- dfs[5]: Top assist provider
- dfs[6]: Inclusions in theoretical teams
- dfs[7]: Selections for international sportsperson of the year
- dfs[8]: International man of the match awards
- dfs[11]: Goalscoring statistics / Club
- dfs[12]: Goalscoring statistics / Country

## Data Transformation

In [96]:
# Omit irrelevant columns
dfs[0] = dfs[0][['Season / year','Competition','Club / national team']]

In [97]:
# Custom function to fill with the last name if the value is a number
df = dfs[0]

# Function to check if a string contains only digits
def is_valid_name(name):
    return not name.isdigit()

# Function to clean the "Club / National team" column
def clean_club_name(row):
    club_name = row["Club / national team"]
    
    # Check if the value is an empty string or not a string
    if not isinstance(club_name, str) or not club_name.strip():
        return club_name
    
    # Use regular expression to find text between square brackets
    match = re.search(r'\[(.*?)\]', club_name)
    
    # If a match is found and it's not a valid name, replace it with a blank
    if match and not is_valid_name(match.group(1)):
        return ""
    
    return club_name

# Apply the cleaning function to the "Club / National team" column
df["Club / national team"] = df.apply(clean_club_name, axis=1)

# Replace empty strings with NaN
df["Club / national team"].replace("", np.nan, inplace=True)

# Fill NaN values with the last non-NaN value from up to down
df["Club / national team"].fillna(method='ffill', inplace=True)

# Delete rows with specified values in the "Competition" column (case-insensitive)
values_to_delete = ["runner-up", "fourth place", "third place", "second place"]
df = df[~df["Competition"].str.contains('|'.join(values_to_delete), case=False)]

# Reset the index after filtering rows
df.reset_index(drop=True, inplace=True)

# Function to assign points based on the "Competition" column
def assign_points(competition):
    if "UEFA Champions League" in competition \
    or "FIFA World Cup" in competition \
    or "Copa América" in competition \
    or "UEFA European Championship" in competition:
        return 3
    else:
        return 1

# Apply the assign_points function to create a new "Points" column
df["Points"] = df["Competition"].apply(assign_points)

# Print the updated DataFrame
df

C:\Users\artur\AppData\Local\Temp\ipykernel_8844\499054203.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Points"] = df["Competition"].apply(assign_points)


,Season / year,Competition,Club / national team,Points
0,2004–05,La Liga,Barcelona,1
1,2005,FIFA U-20 World Cup,Argentina U20,1
2,2005,Supercopa de España,Barcelona,1
3,2005–06,La Liga,Barcelona,1
4,2005–06,UEFA Champions League,Barcelona,3
5,2006,Supercopa de España,Barcelona,1
6,2008,Summer Olympics,Argentina Olympic,1
7,2008–09,La Liga,Barcelona,1
8,2008–09,Copa del Rey,Barcelona,1
9,2008–09,UEFA Champions League,Barcelona,3


In [98]:
# Calculate the sum of points for Cristiano Ronaldo
total_points = df["Points"].sum()

# Create a new DataFrame
summary_df = pd.DataFrame({
    "Player": ["Cristiano Ronaldo"],
    "Collective Title": [total_points]
})

summary_df

,Player,Collective Title
0,Cristiano Ronaldo,56


In [99]:
# Custom function to fill with the last name if the value is a number
df = dfs[2]

# Rename a specific column
df = df.rename(columns={'Organisations[k]': 'Organisations'})

# Omit irrelevant columns
df = df[['Organisations','Category','Edition / year','Result']]

# Delete rows with specified values in the "Competition" column (case-insensitive)
df = df[df["Result"] == 'Won']

# Reset the index after filtering rows
df.reset_index(drop=True, inplace=True)

# Function to assign points based on the "Competition" column
def assign_points(organizations):
    if "Ballon d'Or Gala" in organizations \
    or "FIFA" in organizations:
        return 3
    else:
        return 1

# Apply the assign_points function to create a new "Points" column
df["Points"] = df["Organisations"].apply(assign_points)

# Print the updated DataFrame
df

KeyError: "None of [Index(['Organisations', 'Category', 'Edition / year', 'Result'], dtype='object', name=0)] are in the [columns]"

In [ ]:
# Create a new DataFrame
summary_df["Individual honours"] = df["Points"].sum()

summary_df['Total Honours'] = summary_df['Collective Title'] + summary_df['Individual honours']

summary_df